In [1]:
## Math Library
import os
import pandas as pd
import numpy as np
import math
from scipy.optimize import linear_sum_assignment
from collections import defaultdict, Counter

## Visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Multiprocess
from multiprocessing import Pool
import time

In [2]:
## Define Constants
N_CHILDREN = 1000000
N_GIFT_TYPE = 1000
N_GIFT_QUANTITY = 1000
N_GIFT_PREF = 1000
N_CHILD_PREF = 100
TRIPLETS = 5001
TWINS = 45001

## BLOCK DIM
BLOCK_SIZE = 261
N_BLOCKS = int((N_CHILDREN - TWINS + BLOCK_SIZE - 1) / BLOCK_SIZE)

## Default data source path
INITIAL_SUBMISSION = '../src/twtr.csv'

In [3]:
## Initialize Happiness dictionary
print "INITIALIZE DATA..."
CHILD_PREF = pd.read_csv('../input/child_wishlist_v2.csv', header=None).drop(0, 1).values
GIFT_PREF = pd.read_csv('../input/gift_goodkids_v2.csv', header=None).drop(0, 1).values

print "INITIALIZE GIFT HAPPINESS..."
GIFT_HAPPINESS = {}
for g in range(N_GIFT_TYPE):
    GIFT_HAPPINESS[g] = defaultdict(lambda: -1. / (2 * N_GIFT_PREF))
    for i, c in enumerate(GIFT_PREF[g]):
        GIFT_HAPPINESS[g][c] = 1. * (N_GIFT_PREF - i) / N_GIFT_PREF

print "INITIALIZE CHILD HAPPINESS..."
CHILD_HAPPINESS = {}
for c in range(N_CHILDREN):
    CHILD_HAPPINESS[c] = defaultdict(lambda: -1. / (2 * N_CHILD_PREF))
    for i, g in enumerate(CHILD_PREF[c]):
        CHILD_HAPPINESS[c][g] = 1. * (N_CHILD_PREF - i) / N_CHILD_PREF
print "FINISHED"

INITIALIZE DATA...
INITIALIZE GIFT HAPPINESS...
INITIALIZE CHILD HAPPINESS...
FINISHED


In [4]:
### Function to compute normalized happiness
def my_avg_normalized_happiness(pred):
    total_child_happiness = 0
    total_gift_happiness = np.zeros(1000)
    print "COMPUTE NORMALIZED HAPPINESS..."
    for i, [c,g] in enumerate(pred):
        total_child_happiness +=  CHILD_HAPPINESS[c][g]
        total_gift_happiness[g] += GIFT_HAPPINESS[g][c]
    nch = total_child_happiness / N_CHILDREN
    ngh = np.mean(total_gift_happiness) / 1000
    print('normalized child happiness', nch)
    print('normalized gift happiness', ngh)
    return nch**3. + ngh**3., ngh*N_CHILDREN, nch*N_CHILDREN
    
### Define a new entropy term
def entropy(gh, ch, g, c):
    return 3.*gh*g*(g + gh) + g**3 + 3.*ch*c*(c + ch) + c**3
### Optimize the total entropy
def optimize_block(child_block, gift_block, gh, ch):
    b_size = len(child_block)
    C = np.zeros((b_size, b_size))
    for i,c in enumerate(child_block):
        for j,g in enumerate(gift_block):
            C[i, j] = -1. * entropy(gh, ch, GIFT_HAPPINESS[g][c], CHILD_HAPPINESS[c][g])
    row_ind, col_ind = linear_sum_assignment(C)
    return child_block[row_ind], gift_block[col_ind]
def optimize_wrapper(args):
    return optimize_block(*args)

In [5]:
print("RESTARTING...")
subm = pd.read_csv(INITIAL_SUBMISSION)
initial_anh, g, c = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
print(initial_anh, g, c)

RESTARTING...
COMPUTE NORMALIZED HAPPINESS...
('normalized child happiness', 0.9723309299965189)
('normalized gift happiness', 0.00037143150000000947)
(0.91926833952973452, 371.43150000000946, 972330.9299965189)


In [6]:
from IPython.display import display
display(subm[:7])

,ChildId,GiftId
0,0,200
1,1,200
2,2,200
3,3,245
4,4,245
5,5,245
6,6,791


In [14]:
pool = Pool(8)

In [15]:
t1 = time.time()
gift_idx = subm.GiftId.values
for i in range(20):
    print "=================  Iteration #{0}  =================".format(str(i))
    bsize = 100
    perm = np.random.permutation(range(TWINS, N_CHILDREN))
    child_blocks = [perm[j*bsize: (j+1)*bsize] for j in range(500)]
    args = [(ch_blk, gift_idx[ch_blk], g, c) for ch_blk in child_blocks]
    ans = pool.map(optimize_wrapper, args)
    for cidx, gidx in ans:
        gift_idx[cidx] = gidx
    subm['GiftId'] = gift_idx
    anh, g, c = my_avg_normalized_happiness(subm[['ChildId', 'GiftId']].values.tolist())
    print anh
    print "Time :", time.time() - t1, " s"
    t1 = time.time()
t2 = time.time()
print t2 - t1

=================  Iteration #0  =================
COMPUTE NORMALIZED HAPPINESS...
('normalized child happiness', 0.9723319749965194)
('normalized gift happiness', 0.00037143150000000947)
0.919271303448
Time : 38.8884408474  s
=================  Iteration #1  =================
COMPUTE NORMALIZED HAPPINESS...
('normalized child happiness', 0.9723319749965194)
('normalized gift happiness', 0.00037143150000000947)
0.919271303448
Time : 16.3120470047  s
=================  Iteration #2  =================
COMPUTE NORMALIZED HAPPINESS...
('normalized child happiness', 0.9723319749965194)
('normalized gift happiness', 0.00037143150000000947)
0.919271303448
Time : 24.6120669842  s
=================  Iteration #3  =================
COMPUTE NORMALIZED HAPPINESS...
('normalized child happiness', 0.9723319749965192)
('normalized gift happiness', 0.00037143150000000947)
0.919271303448
Time : 23.7990238667  s
=================  Iteration #4  =================
COMPUTE NORMALIZED HAPPINESS...
('normali

In [13]:
pool.close()
pool.join()

In [8]:
t1 = time.time()
gift_idx = subm.GiftId.values
for i in range(1):
    print "=================  Iteration #{0}  =================".format(str(i))
    bsize = 100
    perm = np.random.permutation(range(TWINS, N_CHILDREN))
    child_blocks = [perm[j*bsize: (j+1)*bsize] for j in range(500)]
    ans = []
    for j in range(500):
        child_block = child_blocks[i]
        gift_block = gift_idx[child_block]
        cids, gids = optimize_block(child_block , gift_block, g, c)
        ans.append((cids, gids))
t2 = time.time()
print t2 - t1

=================  Iteration #0  =================
10.6640958786


In [9]:
print t2 - t1

10.6640958786
